In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
#from imutils import paths
import os
import pandas as pd
import cv2
from random import shuffle
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertConfig
from PIL import Image
import pytesseract
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import zipfile
import tempfile
import string
from PIL import Image #, ImageDraw, ImageFont
#from PIL import ImageOps
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english')) 
tf.test.is_gpu_available(cuda_only=True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

train_df.head(10)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre..."


In [3]:
train_df.shape

(120000, 3)

In [4]:
TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [6]:
def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

In [7]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)
train_df.head()

,Class Index,text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."


In [8]:
train_df['text'][0]

"Wall St. Bears Claw Back Into the Black (Reuters). Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

In [9]:
train_df.rename(columns={'Class Index': 'label'}, inplace=True)
test_df.rename(columns={'Class Index': 'label'}, inplace=True)

In [10]:
train_df.groupby('label').count()

,text
label,
1,30000
2,30000
3,30000
4,30000


In [11]:
train_df['label'] = train_df['label'].replace([1,2,3,4], [0,1,2,3])

In [12]:
train_df.head()

,label,text
0,2,Wall St. Bears Claw Back Into the Black (Reute...
1,2,Carlyle Looks Toward Commercial Aerospace (Reu...
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,2,Iraq Halts Oil Exports from Main Southern Pipe...
4,2,"Oil prices soar to all-time record, posing new..."


In [13]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

In [14]:
train_df['text']=train_df['text'].apply(clean_text)
train_df.head(10)

,label,text
0,2,wall st bears claw back into the black reuters...
1,2,carlyle looks toward commercial aerospace reut...
2,2,oil and economy cloud stocks outlook reuters r...
3,2,iraq halts oil exports from main southern pipe...
4,2,oil prices soar to alltime record posing new m...
5,2,stocks end up but near year lows reuters reute...
6,2,money funds fell in latest week ap ap assets ...
7,2,fed minutes show dissent over inflation usatod...
8,2,safety net forbescom forbescom after earning ...
9,2,wall st bears claw back into the black new yo...


In [15]:
to_txt = list(train_df['text'])

In [16]:
to_txt

['wall st bears claw back into the black reuters reuters  shortsellers wall streets dwindlingband of ultracynics are seeing green again',
 'carlyle looks toward commercial aerospace reuters reuters  private investment firm carlyle groupwhich has a reputation for making welltimed and occasionallycontroversial plays in the defense industry has quietly placedits bets on another part of the market',
 'oil and economy cloud stocks outlook reuters reuters  soaring crude prices plus worriesabout the economy and the outlook for earnings are expected tohang over the stock market next week during the depth of thesummer doldrums',
 'iraq halts oil exports from main southern pipeline reuters reuters  authorities have halted oil exportflows from the main pipeline in southern iraq afterintelligence showed a rebel militia could strikeinfrastructure an oil official said on saturday',
 'oil prices soar to alltime record posing new menace to us economy afp afp  tearaway world oil prices toppling records

In [17]:
#문장 끝에 온점 추가
for i in range(len(to_txt)):
    to_txt[i]=to_txt[i] + " ."

In [18]:
import numpy as np

train_data=np.array(to_txt)

In [19]:
import os
if not os.path.exists("swb/train"):
    os.makedirs("swb/train")
for i in range(0,train_data.shape[0],6):
    text = "\n".join(train_data[i:i+6].tolist())
    fp = open("swb/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

In [20]:
#소문자
from collections import Counter

texts = " ".join(train_data.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("swb/vocab.txt","w")
fp.write(text)
fp.close()

Number of tokens in Training data =  4729896
Size of Vocab 90638


In [21]:
#하이퍼 파라미터 옵션(학습이 끝난 뒤엔 꼭 n_character=262로 해줘야 하므로 재설정. 기본값이 261로 돼있을 것임)
options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 262,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 4096,
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 512,
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': 4729896,
     'batch_size': 64,
     'n_tokens_vocab': 90638,
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [22]:
import json
file_path = "./swb/checkpoint/options.json"
with open(file_path, 'w') as outfile:
    json.dump(options, outfile, indent=4)